In [5]:
import os
from symfit import variables, Parameter, ODEModel, D, Fit, parameters
import numpy as np
import matplotlib.pyplot as plt
from symfit.contrib.interactive_guess import InteractiveGuess2D
%matplotlib tk

In [6]:
from timescan_plot import timescanplot
from time import time as start_time
os.getcwd()

'D:\\FELion-Spectrum-Analyser\\testing\\kinetics\\test'

In [7]:
file = '09_04_19-3.scan'
location = os.getcwd()
time, m, masslist, iterations, t_res, t_b0  = timescanplot(file, location)

for i in masslist:
    plt.plot(time, m[i], label=i)
plt.legend()
plt.show()


ALL: found at line no. 62

#mass24.80:	4

#mass25.80:	4

#mass26.80:	4

#mass27.80:	4

#mass28.80:	4

###############
File: 09_04_19-3.scan
Location: D:\FELion-Spectrum-Analyser\testing\kinetics\test
###############
{'res': 'm03_ao13_reso', 'b0': 'm03_ao09_width'}
{'res': 1.0, 'b0': 10000.0}


In [8]:
plt.plot(time, m['24.8'],'.', label = '24.8')
plt.plot(time, m['25.8'], label = '25.8')
new_sum = m['24.8']+m['25.8']

plt.plot(time, new_sum, 'k', label='sum')
plt.legend()
plt.show()
plt.close()

In [9]:
c2h = m['24.8'][5:]
c2h2 = m['25.8'][5:]
c2h3 = m['26.8'][5:]
c2h4 = m['27.8'][5:]
sum2 = m['24.8'][5:] + m['25.8'][5:] + m['27.8'][5:]
sum3 = m['24.8'][5:] + m['25.8'][5:] + m['26.8'][5:] + m['27.8'][5:]
new_time = time[5:]/1000
plt.plot(new_time, c2h, '.-', label='c2h')
plt.plot(new_time, c2h2, '.-', label='c2h2')
plt.plot(new_time, c2h3, '.-', label='c2h3')
plt.plot(new_time, c2h4, '.-', label='c2h4')
plt.plot(new_time, sum3, '-', label='sum')

In [10]:
C2H, C2H2, C2H3, C2H4, t = variables('C2H, C2H2, C2H3, C2H4, t')
k = Parameter('k', min=1e-14, max=1e-8)
kra = Parameter('kra', min=1e-15, max=1e-11)

initial_cond = {
    t: 0,
    C2H: 9000,
    C2H2: 1,
    C2H4: 1150,
    C2H3: 320,
}

H2 = 1.38e9
rate_model = {
    D(C2H, t): -k*C2H*H2,
    D(C2H2, t): k*C2H*H2-kra*C2H2*H2,
    D(C2H4, t): kra*C2H2*H2,
    D(C2H3, t): kra*C2H2*H2
}

ode_model = ODEModel(rate_model, initial=initial_cond)



In [11]:
# Generate some data
tdata = np.linspace(0, 3, 1000)
K = 3.54e-9
KRA = 1.525e-12

C2H, C2H2, C2H3, C2H4 = ode_model(t=tdata, k=K, kra=KRA)

plt.plot(new_time, c2h, '.', label='c2h_exp')
plt.plot(new_time, c2h2, '.', label='c2h2_exp')
plt.plot(new_time, c2h3, '.', label='c2h3_exp')
plt.plot(new_time, c2h4, '.', label='c2h4_exp')
plt.plot(new_time, sum2, '--', label='sum')

plt.plot(tdata, C2H, '-', label='C2H')
plt.plot(tdata, C2H2, '-', label='C2H2')
plt.plot(tdata, C2H3, '-', label='C2H3')
plt.plot(tdata, C2H4, '-', label='C2H4')

plt.ylabel('Counts')
plt.xlabel('Time (s)')
plt.title(f'k: {K}, kra: {KRA}')
plt.yscale('log')
plt.legend()

In [12]:
t0 = start_time()

fit = Fit(ode_model, t=new_time, C2H=c2h, C2H3=c2h3, C2H2=c2h2)
fit_result = fit.execute()

t1 = start_time()
print(f'Time taken: {t1-t0} s')

print(fit_result)

Time taken: 0.9235379695892334 s

Parameter Value        Standard Deviation
k         2.461197e-09 6.018954e-25
kra       2.461943e-12 1.310562e-25
Fitting status message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
Number of iterations:   87
Regression Coefficient: 0.47987680541106437



C:\ProgramData\Anaconda3\lib\site-packages\symfit\core\fit.py:1046: RuntimeWarning: divide by zero encountered in true_divide
  W = 1/sigma**2/s_sq[:, np.newaxis]


In [ ]:
guess = InteractiveGuess2D(ode_model, t=new_time, C2H=c2h, C2H2=c2h2, C2H3=c2h3, C2H4=c2h4)
guess.execute()